In [ ]:
# FOR RUNNING PURPOSES ONLY - FIXED SEED THROUGHOUT

import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random

from sklearn.cluster import KMeans
from sklearn import metrics

# DATA PROCESSING

header = open('Iris.csv').readlines()[0].strip().split(',')[1:-1]
classes = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
temp = np.array(pd.read_csv('Iris.csv'))
x = temp[:, 1:-1]
y = temp[:, -1]

N = len(x)
M = len(header)

random.seed(278034)

# DATA EVALUATION
# Creating the similarity matrix, adj
# Only edges with a similarity that passes the density threshold will be considered to maintain sparsity of the graph

adj = [[0 for j in range(N)] for i in range(N)]
mn = x.min(axis = 0)
mx = x.max(axis = 0)

ed = []
density = 0.15
P = 30

adj_list = []

for i in range(N):
  for j in range(i+1, N):
    sm = 0
    for h in range(1, M):
      da = (x[i][h] - mn[h]) / (mx[h] - mn[h])
      db = (x[j][h] - mn[h]) / (mx[h] - mn[h])
      sm += abs(da - db)
    sm /= (M - 1)
    ed.append((sm, i, j))

ed.sort()

for i in range(int(density * (N * (N-1) / 2))):
    # print(1 - ed[i][0])
    adj[ed[i][1]][ed[i][2]] = 1 - ed[i][0]
    adj[ed[i][2]][ed[i][1]] = 1 - ed[i][0]
    adj_list.append([ed[i][1], ed[i][2]])

for i in range(int(density * (N * (N-1) / 2)), len(ed)):
  if random.random() < density/P:
    adj[ed[i][1]][ed[i][2]] = (1 - ed[i][0])
    adj[ed[i][2]][ed[i][1]] = (1 - ed[i][0])
    adj_list.append([ed[i][1], ed[i][2]])


# building random points

og_x = []
og_y = []

for i in range(N):
  og_x.append((random.random() - 0.5) * 100);
  og_y.append((random.random() - 0.5) * 100);


# FORCE-DIRECTED GRAPH MODEL

# CONSTANT VARIABLES (to be adjusted for experimentation)
# alpha = 1
# a_decay = 0.1
delta_t = 0.01
iterations = 200

class Node:
  def __init__(self, id, x = 0, y = 0, vx = 0, vy = 0, fx = 0, fy = 0):
    self.id = id
    self.x = x
    self.y = y
    self.vx = vx
    self.vy = vy
    self.fx = fx
    self.fy = fy


  def base_position_recalculation(self):
    # treat fx and fy as displacement

    # update position by adding on displacement vector
    self.x += self.fx * delta_t
    self.y += self.fy * delta_t
    self.fx = 0
    self.fy = 0 

def Gravity(x, y):
  c = 0.1
  xfac = 1
  if x < 0:
    xfac = -1
  yfac = 1
  if y < 0:
    yfac = -1
  return (-c * x * x * xfac, -c * y * y * yfac)

def Linear_Attraction(dx, dy, k):
  c = 1
  mag_d = math.sqrt(dx * dx + dy * dy)
  return (c * k * dx * dx / mag_d, c * k* dy * dy / mag_d)


def Eades_Logarithmic_Attraction(dx, dy, k):
  c = 0.1 # experiment with c
  mag_d = math.sqrt(dx * dx + dy * dy)
  if mag_d == 0:
    return (0, 0)
  return (k * math.log2(mag_d/c) * dx / mag_d, 
          k * math.log2(mag_d/c) * dy / mag_d)

def Inverse_Square_Repulsion(dx, dy, k):
  c = 10 # experiment with c
  mag_d = math.sqrt(dx * dx + dy * dy)
  d = max(mag_d, 1);
  val = c / (d * d)
  return (- val * dx / mag_d, - val * dy / mag_d)

def Standard_Force_Directed_Graph(attract, repulse, gravity):

  nodes = [Node(i) for i in range(N)]
  for i in range(N):
    nodes[i].x = og_x[i]
    nodes[i].y = og_y[i]


  for i in range(iterations):
    for u in range(N):
      for v in range(N):
        if u == v:
          continue
        dx = nodes[v].x - nodes[u].x
        dy = nodes[v].y - nodes[u].y
        if(adj[u][v] > 0):
          af = attract(dx, dy, adj[u][v])
          nodes[u].fx += af[0]
          nodes[u].fy += af[1]
        rf = repulse(dx, dy, 0)
        nodes[u].fx += rf[0]
        nodes[u].fy += rf[1]
      gf = gravity(nodes[u].x, nodes[u].y)
      nodes[u].fx += gf[0]
      nodes[u].fy += gf[1]

    for u in range(N):
      nodes[u].base_position_recalculation()

  return nodes

# def Fruchterman_Reingold_algorithm():
# def LinLog_algorithm():

test_1 = Standard_Force_Directed_Graph(Eades_Logarithmic_Attraction, Inverse_Square_Repulsion, Gravity)

pos_1 = [[test_1[i].x, test_1[i].y] for i in range(N)]



# VISUALISATION SECTION

node_list = pos_1
adj_list = []
for i in range(N):
  for j in range(i+1, N):
    if(adj[i][j] > 0):
      adj_list.append([i, j])



dlist = np.array(node_list)
kmeans = KMeans(n_clusters=3).fit(dlist)
tt = kmeans.predict(dlist)
comp_mat = [[0 for j in range(len(classes))] for i in range(len(classes))]
for i in range(N):
  comp_mat[tt[i]][classes[y[i]]] += 1

best = [0,0,0]
for i in range(len(classes)):
  bst = 0
  for j in range(len(classes)):
    if(comp_mat[i][j] > bst):
      best[i] = j
      bst = comp_mat[i][j]

err = 0

original = []
res = []

def draw_edge(x0, y0, x1, y1): plt.plot([x0, x1], [y0, y1], color='k', markersize = 0, alpha=0.1)
for i in adj_list:
    draw_edge(node_list[i[0]][0], node_list[i[0]][1], node_list[i[1]][0], node_list[i[1]][1])

for i in range(N):
    plt.plot(node_list[i][0], node_list[i][1], marker='o', markersize=4, markeredgecolor=["g","b","r", "m", "y", "k"][classes[y[i]]], markerfacecolor=["g","b","r", "m", "y", "k"][best[tt[i]]])
    # plt.plot(node_list[i][0], node_list[i][1], marker='o', markersize=4, markeredgecolor=["g","b","r", "m", "y", "k"][classes[y[i]]], markerfacecolor=["g","b","r", "m", "y", "k"][classes[y[i]]])
    if classes[y[i]] != best[tt[i]]:
        err += 1
    original.append(classes[y[i]])
    res.append(best[tt[i]])

plt.axis('equal')
plt.show()



print("FDG Accuracy: " + str(1 - err/N))
print("FDG Errors: " + str(err))

# K-means without FDGs

z_kmeans = KMeans(n_clusters=3).fit(x)
z_tt = z_kmeans.predict(x)

z_comp_mat = [[0 for j in range(len(classes))] for i in range(len(classes))]
for i in range(N):
  z_comp_mat[z_tt[i]][classes[y[i]]] += 1

z_best = [0,0,0]
for i in range(len(classes)):
  bst = 0
  for j in range(len(classes)):
    if(z_comp_mat[i][j] > bst):
      z_best[i] = j
      bst = z_comp_mat[i][j]

z_err = 0

z_original = []
z_res = []

for i in range(N):
    if classes[y[i]] != z_best[z_tt[i]]:
        z_err += 1
    z_original.append(classes[y[i]])
    z_res.append(z_best[z_tt[i]])
  
print("Stock K-Means Accuracy: " + str(1 - z_err/N))
print("Stock K-Means Errors: " + str(z_err))

conf_matrix = metrics.confusion_matrix(y_true=original, y_pred=res)

fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()
print(metrics.classification_report(original, res, digits=3))


print('FDG Silhouetter Score: %.3f' % metrics.silhouette_score(X=node_list, labels=kmeans.labels_, metric='euclidean'))
print('FDG Davies-Bouldin Index: %.3f' % metrics.davies_bouldin_score(X=node_list, labels=kmeans.labels_))

print('Stock K-Means Silhouetter Score: %.3f' % metrics.silhouette_score(X=x, labels=z_kmeans.labels_, metric='euclidean'))
print('Stock K-Means Davies-Bouldin Index: %.3f' % metrics.davies_bouldin_score(X=x, labels=z_kmeans.labels_))

conf_matrix = metrics.confusion_matrix(y_true=z_original, y_pred=z_res)

fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

print(metrics.classification_report(z_original, z_res, digits=3))